# ANN with Edge Detection Features

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

### Loading and preprocessing the data

In [2]:
data = pd.read_csv('../svm model/edge_hog_features.csv', header=None)
labels = data.iloc[:, 0].values  # First column: labels
features = data.iloc[:, 1:].values  # Remaining columns: features

# Convert labels to strings (or int if needed)
labels = labels.astype(str)

# Filter out classes with fewer than 2 samples.
label_counts = pd.Series(labels).value_counts()
valid_labels = label_counts[label_counts >= 2].index
filtered_indices = [i for i, label in enumerate(labels) if label in valid_labels]
features = features[filtered_indices]
labels = labels[filtered_indices]

# Impute missing values using mean
imputer = SimpleImputer(strategy='mean')
features = imputer.fit_transform(features)

# Normalize features using StandardScaler
scaler = StandardScaler()
features = scaler.fit_transform(features)

# Convert labels to categorical (first factorize to integers)
labels_int, classes = pd.factorize(labels)
num_classes = len(classes)
labels_cat = to_categorical(labels_int, num_classes=num_classes)

/tmp/ipykernel_64039/3294011660.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('../svm model/edge_hog_features.csv', header=None)


### Spliting Data into Training and Testing Sets

In [3]:
X_train, X_test, y_train, y_test = train_test_split(features, labels_cat, test_size=0.2, random_state=42, stratify=labels)

### ANN Model Definition and training

In [4]:
# ANN Model 
model = Sequential([
    Dense(256, activation='relu', input_shape=(features.shape[1],), kernel_initializer='he_normal'),
    BatchNormalization(),
    Dropout(0.3),
    Dense(128, activation='relu', kernel_initializer='he_normal'),
    BatchNormalization(),
    Dropout(0.3),
    Dense(num_classes, activation='softmax')
])
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

callbacks = [
    EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True),
    ModelCheckpoint("best_ann_model_4.h5", monitor='val_accuracy', save_best_only=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=4, verbose=1)
]

history = model.fit(X_train, y_train,
                    validation_data=(X_test, y_test),
                    epochs=100,
                    batch_size=64,
                    callbacks=callbacks,
                    verbose=1)

# Save final model (if needed)
model.save("best_ann_model_4.h5")
print("Model saved as best_ann_model_4.h5")

/home/sahil-narkhede/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-04-13 12:46:42.074572: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 256)            │     2,073,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 15)             │         1,935 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,110,223 (8.05 MB)

 Trainable params: 2,109,455 (8.05 MB)

...

15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 62ms/step - accuracy: 0.9938 - loss: 0.0755 - val_accuracy: 0.8208 - val_loss: 0.6278 - learning_rate: 0.0010
Epoch 9/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 67ms/step - accuracy: 0.9932 - loss: 0.0585 - val_accuracy: 0.8208 - val_loss: 0.6136 - learning_rate: 0.0010
Epoch 10/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.9922 - loss: 0.0592

15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 64ms/step - accuracy: 0.9922 - loss: 0.0590 - val_accuracy: 0.8292 - val_loss: 0.6255 - learning_rate: 0.0010
Epoch 11/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.9965 - loss: 0.0475

15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 60ms/step - accuracy: 0.9967 - loss: 0.0472 - val_accuracy: 0.8375 - val_loss: 0.6414 - learning_rate: 0.0010
Epoch 12/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 67ms/step - accuracy: 1.0000 - loss: 0.0310 - val_accuracy: 0.8375 - val_loss: 0.6364 - learning_rate: 0.0010
Epoch 13/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.9971 - loss: 0.0346


Epoch 13: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 55ms/step - accuracy: 0.9970 - loss: 0.0347 - val_accuracy: 0.8417 - val_loss: 0.6265 - learning_rate: 0.0010
Epoch 14/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 48ms/step - accuracy: 0.9977 - loss: 0.0304 - val_accuracy: 0.8417 - val_loss: 0.6255 - learning_rate: 5.0000e-04
Epoch 15/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 61ms/step - accuracy: 1.0000 - loss: 0.0234 - val_accuracy: 0.8375 - val_loss: 0.6201 - learning_rate: 5.0000e-04
Epoch 16/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 51ms/step - accuracy: 0.9999 - loss: 0.0197 - val_accuracy: 0.8333 - val_loss: 0.6159 - learning_rate: 5.0000e-04
Epoch 17/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 49ms/step - accuracy: 0.9998 - loss: 0.0171 - val_accuracy: 0.8333 - val_loss: 0.6126 - learning_rate: 5.0000e-04
Epoch 18/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 49ms/step - accuracy: 0.9996 - loss: 0.0206 - val_accuracy: 0.8333 - val_loss: 0.6129 - learning_rate: 5.0000e-

Model saved as best_ann_model_4.h5


### Evaluation

In [7]:
# Evaluating the model on training set
train_loss, train_acc = model.evaluate(X_train, y_train, verbose=0)
print("Training Accuracy:", train_acc)

# Evaluating the model on test set
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy: {test_acc:.2f}")

y_pred = model.predict(X_test).argmax(axis=1)
y_true = y_test.argmax(axis=1)

print("\nClassification Report:")
print(classification_report(y_true, y_pred))

Training Accuracy: 1.0
Test Accuracy: 0.83
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step

Classification Report:
              precision    recall  f1-score   support

           0       0.62      0.62      0.62        16
           1       0.63      0.75      0.69        16
           2       0.93      0.81      0.87        16
           3       1.00      0.88      0.93        16
           4       0.90      0.56      0.69        16
           5       0.80      1.00      0.89        16
           6       0.71      0.75      0.73        16
           7       1.00      0.62      0.77        16
           8       0.71      0.94      0.81        16
           9       0.71      0.62      0.67        16
          10       0.94      1.00      0.97        16
          11       0.89      1.00      0.94        16
          12       1.00      1.00      1.00        16
          13       0.84      1.00      0.91        16
          14       1.00      0.94      0.97        16

    accuracy                 